In [158]:
import torch
import torch.cuda
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [159]:
# Load and normalizde the data

transform = transforms.Compose(
    [transforms.Resize(64),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batchSize = 100
validSize = 0.2 # use 20% of train set as validation

trainValidSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainSet, validSet = torch.utils.data.random_split(trainValidSet, [int(len(trainValidSet)*(1-validSize)), int(len(trainValidSet)*validSize)])

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True)
validLoader = torch.utils.data.DataLoader(validSet, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {'cuda' if torch.cuda.is_available() else 'cpu'}")

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
Using cuda


In [160]:
# Writer for tensorBoard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [161]:
def trainNetwork(network, optimizer, lossFunction, trainLoader, validLoader, epochs, device, writer, name):
    network.train()
    for epoch in range(epochs):
        
        ### TRAINING ###
        trainLoss = 0
        correctTrain = 0
        totalTrain = 0
        for batch_nr, (images, labels) in enumerate(trainLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  

            # Predict
            predictions = network(images)

            # Get loss and backpropogate
            loss = lossFunction(predictions, labels)
            loss.backward() 

            # Optimize parameters (weights and biases) and remove gradients after
            optimizer.step() 
            optimizer.zero_grad()

            # Save loss for whole epoch
            trainLoss += loss.item()
            
            # Calculate training accuracy
            _, predictions = torch.max(predictions, 1) 
            correctTrain += (predictions == labels).sum().item() 
            totalTrain += len(images)

            #print(f'Epoch [{epoch+1}/{epochs}] Batch [{batch_nr}/{len(trainLoader)}]')
        

        trainLoss /= len(trainLoader)
        trainAccuracy = 100 * correctTrain / totalTrain
        writer.add_scalar(f"Loss/train - {name}", trainLoss, epoch)
        writer.add_scalar(f"Accuracy/train - {name}", trainAccuracy, epoch)

        ### VALIDATION ###
        validLoss = 0
        correctValid = 0
        totalValid = 0
        for batch_nr, (images, labels) in enumerate(validLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  

            # Predict
            predictions = network(images)

            # Get loss
            loss = lossFunction(predictions, labels)

            # Save loss for whole epoch
            validLoss += loss.item()

            # Calculate vaildation accuracy
            _, predictions = torch.max(predictions, 1) 
            correctValid += (predictions == labels).sum().item() 
            totalValid += len(images)

            #print(f'Epoch [{epoch+1}/{epochs}] Batch [{batch_nr}/{len(validLoader)}]')

        validLoss /= len(validLoader)
        validAccuracy = 100 * correctValid / totalValid
        writer.add_scalar(f"Loss/valid - {name}", validLoss, epoch)
        writer.add_scalar(f"Accuracy/valid - {name}", validAccuracy, epoch)

        # Print reuslt of epoch
        print(f'Epoch [{epoch+1}/{epochs}] \t Training Loss: {trainLoss} \t Validation Loss: {validLoss} \t Traning Acc: {trainAccuracy}% \t Validation Acc: {validAccuracy}%')

    writer.flush()

In [162]:
def testNetwork(network, testLoader, device):
    network.eval()
    correctTest = 0
    totalTest = 0

    ### TESTING ###
    with torch.no_grad(): 
        for batch_nr, (images, labels) in enumerate(testLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  
            
            # Get predictions and get the amount of correct predicitons
            predictions = network(images)
            _, predictions = torch.max(predictions, 1) 
            correctTest += (predictions == labels).sum().item() 
            totalTest += len(images)
            
            #print(f'Batch [{batch_nr}/{len(testLoader)}]', end='\r')
        
    print(f"Test Accuracy: {100 * correctTest / totalTest}%")

In [199]:
class myAlexNet(nn.Module):
    def __init__(self, pretrained):
        super(myAlexNet,self).__init__()
        self.alexNet = torchvision.models.alexnet(pretrained=pretrained)
        self.out = nn.Linear(1000,10)
    
    def forward(self, x):
        x = F.relu(self.alexNet.forward(x))
        # x = self.alexNet.forward(x)
        return self.out(x)


In [200]:
learningRate = 0.01
epochs = 10

In [201]:
networkNotTrained = myAlexNet(pretrained=False).to(device)

In [202]:
optimizerAdam = torch.optim.Adam(networkNotTrained.parameters(), lr=learningRate)
lossFunction = nn.CrossEntropyLoss().to(device)

In [203]:
# networkNotTrained.eval()

In [204]:
trainNetwork(networkNotTrained, optimizerAdam, lossFunction, trainLoader, validLoader, epochs, device, writer, "Not pretrained")

Epoch [1/10] 	 Training Loss: 2.3027343833446503 	 Validation Loss: 2.3025461900234223 	 Traning Acc: 9.8975% 	 Validation Acc: 9.96%
Epoch [2/10] 	 Training Loss: 2.3025175839662553 	 Validation Loss: 2.302488067150116 	 Traning Acc: 10.43% 	 Validation Acc: 11.0%
Epoch [3/10] 	 Training Loss: 2.3023233672976495 	 Validation Loss: 2.30221564412117 	 Traning Acc: 11.0775% 	 Validation Acc: 14.7%
Epoch [4/10] 	 Training Loss: 2.3019089189171793 	 Validation Loss: 2.3016091001033785 	 Traning Acc: 12.8% 	 Validation Acc: 14.53%
Epoch [5/10] 	 Training Loss: 2.300174323916435 	 Validation Loss: 2.297221142053604 	 Traning Acc: 14.115% 	 Validation Acc: 12.5%
Epoch [6/10] 	 Training Loss: 2.236694769859314 	 Validation Loss: 2.1272905433177947 	 Traning Acc: 14.22% 	 Validation Acc: 16.97%
Epoch [7/10] 	 Training Loss: 2.0388277776539327 	 Validation Loss: 1.9734901297092438 	 Traning Acc: 18.525% 	 Validation Acc: 19.61%
Epoch [8/10] 	 Training Loss: 1.9544938750565053 	 Validation Loss: 

In [213]:
testNetwork(networkNotTrained, testLoader, device)

Test Accuracy: 26.5%


In [208]:
networkPretrained = myAlexNet(pretrained=True).to(device)

for param in networkPretrained.alexNet.parameters():
    param.requires_grad=False
 
optimizerAdam = torch.optim.Adam(networkPretrained.parameters(), lr=learningRate)

In [209]:
trainNetwork(networkPretrained, optimizerAdam, lossFunction, trainLoader, validLoader, epochs, device, writer, "Pretrained weights")

Epoch [1/10] 	 Training Loss: 3.515480038970709 	 Validation Loss: 3.0580851513147356 	 Traning Acc: 42.0475% 	 Validation Acc: 45.58%
Epoch [2/10] 	 Training Loss: 2.912719173580408 	 Validation Loss: 2.5724073594808576 	 Traning Acc: 47.4075% 	 Validation Acc: 48.3%
Epoch [3/10] 	 Training Loss: 2.852765713632107 	 Validation Loss: 2.761557629108429 	 Traning Acc: 48.1475% 	 Validation Acc: 47.51%
Epoch [4/10] 	 Training Loss: 2.7217942439764737 	 Validation Loss: 2.8112743306159973 	 Traning Acc: 49.0% 	 Validation Acc: 48.83%
Epoch [5/10] 	 Training Loss: 2.676224146038294 	 Validation Loss: 2.448267377614975 	 Traning Acc: 49.5125% 	 Validation Acc: 51.19%
Epoch [6/10] 	 Training Loss: 2.7117543345689774 	 Validation Loss: 2.502414426803589 	 Traning Acc: 49.125% 	 Validation Acc: 48.19%
Epoch [7/10] 	 Training Loss: 2.6368246693909168 	 Validation Loss: 3.384044174551964 	 Traning Acc: 49.98% 	 Validation Acc: 47.22%
Epoch [8/10] 	 Training Loss: 2.59664536036551 	 Validation Los

In [210]:
testNetwork(networkPretrained, testLoader, device)

Test Accuracy: 50.88%


In [211]:
# # Load pre-trained AlexNet model
# alexnet = torchvision.models.alexnet(pretrained=True)

# # Replace last layer with new layer with 10 output classes
# alexnet.classifier[6] = torch.nn.Linear(4096, 10)

# # Freeze all layers except the last layer
# for param in alexnet.parameters():
#    param.requires_grad = False
# for param in alexnet.classifier[6].parameters():
#     param.requires_grad = True

# alexnet.to(device)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [212]:
# trainNetwork(alexnet, optimizerAdam, lossFunction, trainLoader, validLoader, epochs, device, writer, "Pretrained weights")


Epoch [1/10] 	 Training Loss: 2.8639135611057283 	 Validation Loss: 2.863082581758499 	 Traning Acc: 8.165% 	 Validation Acc: 8.21%
Epoch [2/10] 	 Training Loss: 2.866484526991844 	 Validation Loss: 2.8515813863277435 	 Traning Acc: 8.12% 	 Validation Acc: 8.14%
Epoch [3/10] 	 Training Loss: 2.8605616164207457 	 Validation Loss: 2.873180342912674 	 Traning Acc: 8.2225% 	 Validation Acc: 7.99%
Epoch [4/10] 	 Training Loss: 2.8626927369832993 	 Validation Loss: 2.8723754334449767 	 Traning Acc: 8.0925% 	 Validation Acc: 7.97%


KeyboardInterrupt: ignored